# Constructing models for the Task of Microblog Opinion Summarisation
----------

We will use use the Microblog Opinion Summarisation corpus described in https://arxiv.org/pdf/2208.04083.pdf .

#Table of contents

>[Constructing models for the Task of Microblog Opinion Summarisation](#scrollTo=PGGvuJqQkotL)

>>[Table of contents](#scrollTo=U_Dzawo3k2YQ)

>>[Prepare your environment](#scrollTo=dKE_mAtIk9u8)

>>[Construction of the gold standard summary](#scrollTo=TKXz8EbKOc9g)

>>[Summarisation Model and Fine-tuning](#scrollTo=JYEsFM58Oc9i)

>>[Generated Summary Evaluation](#scrollTo=RBlNZjywOc9k)

#Prepare your environment

In [1]:
!pip install datasets
!pip install sentence-transformers
!pip install transformers
!pip install -r rouge/requirements.txt
!pip install rouge-score
!pip install bert_score
!pip install tabulate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 13.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [20]:
import json
import os
import numpy
import re
import shutil
import tabulate
import nltk
nltk.download('punkt')

from transformers import BartForConditionalGeneration, AutoModel, AutoTokenizer, DataCollatorForSeq2Seq 
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from sentence_transformers import SentenceTransformer, util
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer
from bert_score import score
from datasets import load_dataset


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# connect to gdrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
dir_path = '/content/gdrive/MyDrive/PhD/Code/mos_workspace/'

Most sequence-to-sequence models have input limit of 512/1024 tokens.

Order the tweets within a cluster by relevance. 
We use sentence-tranformers library from Hugging Face to embed the tweets in a vector space and then use the cosine similarity as proxy for a tweet's relevance in the cluster. This can be done in two ways:
- <i>Medoid</i>: For each tweet, compute the mean distance to all other tweets in the vector space. Find the tweet (medoid) which minimises this. As an alternative to cosine similarity, one can use Euclidean distance.
- <i>Centroid</i>: Compute the centroid of the cluster (which usually does not exists in practice) as the mean average of their embeddings. Then find the closest tweet to the centroid as the centroid approximation.

In [17]:
def cluster_distance(tweet_embeddings,tweet_emb):
    distances = [1-util.pytorch_cos_sim(tweet_emb, tweet_emb2) for tweet_emb2 in tweet_embeddings]
    return sum(distances)

def cluster_distance2(tweet_embeddings,tweet_emb):
    distances = [numpy.linalg.norm(tweet_emb- tweet_emb2) for tweet_emb2 in tweet_embeddings]
    return sum(distances)  

def centroid_distance(tweet_embeddings, centroid):
    return 1- util.pytorch_cos_sim(tweet_embeddings, centroid)



In our preprocessing, we remove emojis, URL placeholders and whitespace characters.

In [15]:
#Preprocess tweets
def preprocess_(tweets):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        "]+", flags=re.UNICODE)
    
    tweets = [emoji_pattern.sub(r'', tweet) for tweet in tweets]
    tweets = [tweet.replace('\n', ' ') for tweet in tweets]
    tweets = [tweet.replace('\r', ' ') for tweet in tweets]
    tweets = [tweet.replace('\t', ' ') for tweet in tweets]
    tweets = [tweet.replace('URL_LINK','') for tweet in tweets]
   
    return tweets

tweet = 'Hello World!😄 \nWhat a great day to learn! Check this URL_LINK'
print(tweet)
print(preprocess_([tweet]))    

Hello World!😄 
What a great day to learn! Check this URL_LINK
['Hello World!  What a great day to learn! Check this ']


In [18]:

k = 7
model_st = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def sort_relevance(cluster):
    tweets = []
    minv = 100000 
    pp_tweets = preprocess_(cluster['tweets'])
    tweet_embeddings = model_st.encode(pp_tweets)
    centroid = sum(tweet_embeddings)/len(tweet_embeddings)

    for i, tweet in enumerate(pp_tweets):
        tweets.append({'tweet':tweet, 'score': cluster_distance(tweet_embeddings,tweet_embeddings[i])})

    tweets = sorted(tweets, key=lambda x: x['score'])
    return [x['tweet'] for x in tweets]

with open(dir_path + '/testing_set/opi_elections/2014-08-25-22_homeless_0') as f:
    cluster = json.load(f)

print('The most relevant {} tweets in the cluster are:'.format(k), sort_relevance(cluster)[:k])   







The most relevant 7 tweets in the cluster are: ['I actually think it was lovely that Miley Cyrus asked a homeless guy to accept the award on her behalf! Using her fame for good! #VMAs', 'miley cyrus brought an homeless guy with her to the VMAs aww @MileyCyrus', "Not a Miley Cyrus fan in anyway, but allowing a homeless guy at the VMA's to accept her award is very noble of her. My hat goes off to her!", 'Miley Cyrus sent a homeless man, who she brought as her date, up to accept her award #awesome', 'Fair play to miley for getting a homeless guy to collect her VMA award on her behalf', 'What miley did was so inspiring to let the homeless man take the award for her ♥', 'okay I have respect for Miley Cyrus for letting a homeless man accept her award']


# Construction of the gold standard summary

Our chosen summary structure diverges from current summarisation approaches that reconstruct the “most popular opinion” (Bražinskas et al., 2020; Angelidis et al., 2021). Instead, we aim to showcase a spectrum of diverse opinions regarding the same event as this is a rich resource for practitioners. Thus, the summary template comprises
three components: 

* **Main Story**: serves to succinctly present the focus of the cluster (often an event)
* **Majority Opinion**: the opinion expressed by most posts in the cluster regarding the main story
* **Minority Opinion(s)**: opinions expressed by few posts in the cluster regarding the main story. 

For the purpose of our project, the gold standard summary is the concatenation of all components: main story, majority opinion (if any) and minority opinions (if any). 

We define a cluster with many opinions as *opinionated* and a cluster with mostly factual information (and few or no opinions) as *non-opinionated*. 


In [ ]:

def summary_concat(cluster):
    opinionated = 1
    if 'majority_opinion' in cluster.keys() and 'minority_opinions' in cluster.keys():
        summary = cluster['main_story'] + ' ' + cluster['majority_opinion'] + ' ' + cluster['minority_opinions']
    elif 'majority_opinion' in cluster.keys() and 'minority_opinions' not in cluster.keys():  
        summary = cluster['main_story'] + ' ' + cluster['majority_opinion']
    elif 'majority_opinion' not in cluster.keys() and 'minority_opinions' in cluster.keys():
        summary = cluster['main_story'] + ' ' + cluster['minority_opinions']   
    else:
       summary = cluster['main_story']  
       opinionated = 0
    return summary, opinionated 

We save the clusters with the ordered tweets. Note that the cell below is optional to run as the saved input file *partition10_relevance.json* is provided in the workspace folder.

In [ ]:
#Save the clusters with ordered input
list_clusters = []
for file in os.listdir(dir_path + 'Partition_10'):
    if not os.path.isfile(os.path.join(dir_path + 'Partition_10',file)):
        continue
    with open(os.path.join(dir_path + 'Partition_10',file),'r') as f:
        cluster = json.load(f)
    
    list_clusters.append({'text':' '.join(sort_relevance(cluster)), 'summary':summary_concat(cluster)[0]})
       
with open(dir_path + 'partition10_relevance.json','w') as g:
    for cluster in list_clusters:
        g.write(json.dumps(cluster)+'\n')




We calculate the average length of the gold standard summaries. This will later inform our summary generation step.

In [28]:
avg_len_opi = 0
avg_len_nopi = 0
for file in os.listdir(dir_path + 'Partition_10'):
    if not os.path.isfile(os.path.join(dir_path + 'Partition_10',file)):
        continue
    with open(os.path.join(dir_path + 'Partition_10',file),'r') as f:
        cluster = json.load(f)
    
    if summary_concat(cluster)[1] == 0:
        avg_len_nopi += len(word_tokenize(summary_concat(cluster)[0]))  
    
    else:
        
        avg_len_opi += len(word_tokenize(summary_concat(cluster)[0]))  


avg_len_opi = avg_len_opi/len(os.listdir(dir_path + 'Partition_10/opinionated'))

avg_len_nopi = avg_len_nopi/len(os.listdir(dir_path + 'Partition_10/non_opinionated'))



print('Opinionated summaries are {:2} tokens long on avg'.format(avg_len_opi))
print('Non-Opinionated summaries are {:2} tokens long on avg'.format(avg_len_nopi))




Opinionated summaries are 37.8859649122807 tokens long on avg
Non-Opinionated summaries are 18.41 tokens long on avg


# Summarisation Model and Fine-tuning

We work with Transformer-based model BART for the summarisation task. Its paper can be found here https://arxiv.org/pdf/1910.13461.pdf 
The model produces competitive results across many tasks and domains and it is especially effective for fine-tuning strategies.
We follow the implementation from HuggingFace library and use both its off-the-shelf version as well as fine-tuned version for our task.

BART is fine-tuned for 5 epochs on the 10% partition of the original MOS corpus. The data is stored in file *partition10_relevance.json*. The final model will be saved in the folder ./mos_workspace/BART_model.

Note that we use the <b>Trainer</b> class for ease of use. 'Trainer is a simple but feature-complete training and eval loop for PyTorch, optimized for 🤗 Transformers.' More details can be found at https://huggingface.co/docs/transformers/main_classes/trainer .



In [31]:
#Load the off-the-shelf BART model

model_name = 'facebook/bart-large-cnn'
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
#Set the length limits for each summary type: opinionated and non-opinionated clusters.
length_limits= {'nopi':[14,22],'opi':[30,45]}

def bart_zero(cluster,max_length,min_length):
    original_text = ' '.join(sort_relevance(cluster))
    inputs = tokenizer.encode(original_text, return_tensors="pt", max_length=1024)
    outputs = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True,no_repeat_ngram_size=4)

    summary = tokenizer.decode(outputs[0],skip_special_tokens=True)
    return summary

with open(dir_path + '/testing_set/opi_elections/2014-08-25-22_homeless_0') as f:
    cluster = json.load(f)
print(bart_zero(cluster,length_limits['opi'][1],length_limits['opi'][0]))


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


I actually think it was lovely that Miley Cyrus asked a homeless guy to accept the award on her behalf! Using her fame for good! #VMAs miley cyrus brought an homeless guy with her to


In [32]:
def preprocess_function(examples):
    inputs = examples["text"]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Fine-tune the summarisation model on our dataset. Note that running this cell is optional,  a fine-tuned version of the model can be found [here](https://drive.google.com/drive/folders/1cUrIDM4C6vVM1Rha0UJ9Pg6Ds33cUg33?usp=sharing) (you need to download the entire folder you are directed to in the link; download it and put it in your workspace folder.).

In [ ]:
training_args = TrainingArguments(
    output_dir= dir_path + 'BART_model',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir= dir_path + 'BART_model/logs',            # directory for storing logs
    logging_steps=10,
)


datafiles =dir_path + 'partition10_relevance.json'
dataset = load_dataset('json', data_files=datafiles, split='train')
tokenized_dataset = dataset.map(preprocess_function,batched=False)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_dataset,         # training dataset
    tokenizer = tokenizer,
    data_collator=data_collator,
)

trainer.train()
trainer.save_state()
trainer.save_model()

In [37]:
model_name_ft = dir_path +'BART_model'
model_ft = BartForConditionalGeneration.from_pretrained(model_name_ft)
tokenizer_ft = AutoTokenizer.from_pretrained(model_name_ft)


def bart_ft(cluster,max_length,min_length):
    original_text = ' '.join(sort_relevance(cluster))
    inputs = tokenizer_ft.encode(original_text, return_tensors="pt", max_length=1024)
    outputs = model_ft.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True,no_repeat_ngram_size=4)

    summary = tokenizer_ft.decode(outputs[0],skip_special_tokens=True)
    return summary
print(bart_ft(cluster,length_limits['opi'][1],length_limits['opi'][0]))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Miley Cyrus brought a homeless man on stage to accept Video of the Year at the VMA Awards. The majority think it was good that Miley Cyrus allowed the homeless man to accept the award


# Generated Summary Evaluation

The generated summaries are evaluated against two dimensions:

1. <b>Word-overlap</b>: ROUGE (Lin et al.,2004, https://aclanthology.org/W04-1013.pdf) is the most used metric in summarisation tasks. It calculates the n-gram overlap between system and candidate summaries. In our experiment, we use the harmonic mean F_1 of ROUGE-1 (unigrams), ROUGE-2 (bigrams) and ROUGE-L (longest sequence). By design, the metric is unable to detect semantic similarity (<i>happy kid</i> and <i>joyful child</i> are identified as disjoint and are penalised).

2. <b>Semantic Similarity</b>: BERTScore (Zhang et al, 2020, https://arxiv.org/pdf/1904.09675.pdf) is used in machine generation tasks to detect the level of semantic similarity between a pair of texts. It relies on BERT contextual embeddings and greedy token matching and it has been shown to correlate well with human judgement. It achieved competitive results in machine translation and image captioning due to its robustness in paraphrasing. Note that the metric tends to have a narrow score range between [0.70, 1.00].

In [ ]:
# ROUGE & BERTScore Evaluation

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeL'], use_stemmer=True)
testing_set_path = dir_path + 'testing_set'

for partition in ['nopi_covid','nopi_elections','opi_covid','opi_elections']:
    rouge1_score = {'medoid': 0, 'bart_zero':0, 'bart_ft':0}
    rouge2_score = {'medoid': 0, 'bart_zero':0, 'bart_ft':0}
    rougel_score = {'medoid': 0, 'bart_zero':0, 'bart_ft':0}
    bertscore_score = {'medoid': 0, 'bart_zero':0, 'bart_ft':0} 
    
    for file in os.listdir(os.path.join(testing_set_path,partition)):
        if file[0]=='.':
            continue
        print(file)
        with open(os.path.join(testing_set_path,partition,file),'r') as f:
            cluster = json.load(f)
    
        gold_standard = summary_concat(cluster)[0]
        
        #Generate the summary candidates
        medoid_summary = sort_relevance(cluster)[0]
        
        min_length = length_limits[partition.split('_')[0]][0]
        max_length = length_limits[partition.split('_')[0]][1]
        bart_zero_summary = bart_zero(cluster,max_length,min_length)
        bart_ft_summary = bart_ft(cluster,max_length,min_length)
        summaries = {'medoid':medoid_summary, 'bart_zero':bart_zero_summary, 'bart_ft':bart_ft_summary}
        print(summaries)
        #For each summary type, compute its scores vs the human-written summary
        for key in summaries.keys():
        
            scores = scorer.score(gold_standard, summaries[key])

            rouge1_score[key] += scores['rouge1'][2]
            rouge2_score[key] += scores['rouge2'][2]
            rougel_score[key] += scores['rougeL'][2]
            
            P, R, F1 = score([gold_standard], [summaries[key]], lang='en', verbose=False)
            bertscore_score[key] += F1
    print('Results for partition: ',partition)
    data = [['Model','ROUGE-1', 'ROUGE-2','ROUGE-L', 'BERTScore']]
    for key in summaries.keys():
        data.append([key,rouge1_score[key]/25,rouge2_score[key]/25,rougel_score[key]/25,bertscore_score[key]/25])
    table = tabulate.tabulate(data)  
    print(table)
            


2020-10-01-03_social_distancing_0
{'medoid': '@realdonaldtrump every nurse & physician treating covid patients wants to ask why you continue to neglectfully hold crowded  rallies with no mask mandate and no social distancing. you do not care if people catch covid there. and expose others. what a nightmare for contact tracers. you lost votes! ', 'bart_zero': '@realdonaldtrump every nurse & physician treating covid patients wants to ask why you', 'bart_ft': 'Debate continues about whether or not people should have to observe social distancing and'}


Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

...

2015-03-18-13_military_1
{'medoid': '£75m help to Regimental charities, military museums & Iraq memorial #Budget2015', 'bart_zero': '£75m help to Regimental charities, military museums & Iraq memorial #Budget2015 RT @BBCBusiness: Help for former military service personnel and regimental charities. Increased support for services charities and military museums', 'bart_ft': 'Budget 2015 includes extra £75m for military charities and automatic gift aid for ex-servicemen and women The majority are delighted to see the Chancellor is backing military charities and veterans.'}


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaM

2016-04-04-16_tax_avoidance_1
{'medoid': 'Independent:  The Panama papers tax avoidance scandal shows just how powerless Cameron and… ', 'bart_zero': "David Cameron embroiled in tax avoidance row after details of father's business interests are leaked. Cameron's dad guilty of industrial tax avoidance that paid for Cameron's Eton education and it's perks... In 2012 David Cameron", 'bart_ft': "Details of David Cameron's father's business interests in the Bahamas are leaked in the Panama Papers tax avoidance scandal. The scandal shows just how powerless Cameron and Osborne are in relation to tax avoidance, despite"}


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaM

2015-12-15-22_schools_0


### Results for partition:  non-opinionated covid
------------------------------------
Model   |   ROUGE-1       |       ROUGE-2       |       ROUGE-L    |          BERTScore
--- | --- | --- | --- | ---
**Medoid**   |  0.2485 | 0.0576 | 0.1874 |0.8485
**BART (non-finetuned)** | 0.2088 | 0.0626 | 0.1777  | 0.8461
**BART (finetuned)**  |  0.3166 |  0.1153 | 0.2671  | 0.8782


###Results for partition:  non-opinionated elections
---------
Model    |  ROUGE-1       |       ROUGE-2      |        ROUGE-L    |          BERTScore
--- | --- | --- | --- | ---
**Medoid**   |  0.3735 |  0.1581 | 0.2722  |0.8855
**BART (non-finetuned)** | 0.3849 | 0.1676 |  0.2963  | 0.8970
**BART (finetuned)**  |  0.4260  | 0.2063 | 0.3467  | 0.9068


### Results for partition:  opi_covid
-----------------
Model   |   ROUGE-1       |       ROUGE-2        |      ROUGE-L    |          BERTScore
--- | --- | --- | --- | ---
**Medoid**  |   0.2885  | 0.1300 |  0.2340 |  0.8553
**BART (non-finetuned)** | 0.3090  | 0.1319 |  0.2412 |  0.8575
**BART (finetuned)**  |  0.4155 | 0.1896 | 0.3224 | 0.8922
